In [1]:
import folium
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import json

Import the Census Zipcode Boundaries

In [2]:
census_zips = pd.read_csv('https://raw.githubusercontent.com/jlian014/yelp/master/yelp_geo_zips.csv')

Import a cleaned yelp dataset

In [3]:
yelp_with_dollars = pd.read_csv('https://raw.githubusercontent.com/jlian014/yelp/master/daniel/Yelp_Clean.csv')

Specify a range in the dataset that you want to plot. For this example, it will be the zipcodes: 85255,85377,85266,85331,85263,85262

In [4]:
specific_zips = [85255,85377,85266,85331,85263,85262]

In [5]:
specific_frame = yelp_with_dollars[yelp_with_dollars['zip'].isin(specific_zips)]

In [6]:
specific_frame.reset_index(drop=True,inplace=True)

Import and clean the census zipcode boundaries

Download the file here: https://github.com/jlian014/yelp/blob/master/daniel/cb_2017_us_cbsa_20m.shp?raw=true

In [7]:
census_zips_bounds = gpd.read_file('/Volumes/data/cb_2017_us_zcta510_500k/cb_2017_us_zcta510_500k.shp')

In [8]:
census_zips_bounds = census_zips_bounds[['ZCTA5CE10','geometry']]
census_zips_bounds.columns = ['zip','geometry']
census_zips_bounds['zip'] = census_zips_bounds['zip'].astype(int)
census_frame = pd.DataFrame(census_zips_bounds)

Filter the census zipcode boundaries to the ones selected

In [9]:
filtered_zips = census_frame[census_frame['zip'].isin(specific_zips)]
filtered_zips.reset_index(drop=True,inplace=True)

In [10]:
filtered_census = census_zips_bounds[census_zips_bounds['zip'].isin(filtered_zips['zip'])]
filtered_census.reset_index(drop=True,inplace=True)
filtered_census.crs = {'init' :'epsg:4326'}

Import IRS zipcode influence and merge with geometry data

In [11]:
zip_stats = pd.read_csv('https://raw.githubusercontent.com/jlian014/yelp/master/IRS_zipcodes_clean.csv')
zip_stats.columns = ['state','zip',1,2,3,4,5,6]
zip_stats_tidy = filtered_census.merge(zip_stats,on='zip',how='left')[['zip',1,2,3,4,5,6]]

Add Affluence Stats (i arbitrarily created some stats, but you can use whichever you want)

In [12]:
zip_stats['total'] = zip_stats[[1,2,3,4,5,6]].sum(axis=1)

In [14]:
zip_stats['affluence'] = ((zip_stats[1] * 1) 
                          + (zip_stats[2] * 2) 
                          + (zip_stats[3] * 3) 
                          + (zip_stats[4] * 4) 
                          + (zip_stats[4] * 4)
                          + (zip_stats[5] * 5)
                          + (zip_stats[6] * 6) / zip_stats['total'])

In [15]:
zip_stats_supertidy = zip_stats[['zip','affluence']]
filtered_census = filtered_census.merge(zip_stats_supertidy,on='zip',how='left')

In [16]:
filtered_census['zip'] = filtered_census['zip'].astype(str)
tidy_census = filtered_census[['zip','affluence']]

In [17]:
tidy_census.columns = ['regionid','stat']
filtered_census.columns = ['regionid','geometry','stat']
filtered_census.crs = {'init' :'epsg:4326'}

### Now the Chloropleth can be plotted

In [18]:
m = folium.Map([34, -112], zoom_start=8)

In [19]:
chloropleth = folium.Choropleth(
    geo_data=filtered_census,
    data=tidy_census,
    columns=['regionid', 'stat'],
    key_on='feature.properties.regionid',
    fill_color='YlGnBu',
    highlight=True,
    name='Neighborhood Affluency',
    #bins=[3, 4, 9, 11]
).add_to(m)

In [20]:
m

Save the file out

In [ ]:
m.save(outfile='test_chloro_jan16.html')

Plot individual points

In [21]:
specific_frame1 = specific_frame.loc[specific_frame['dollar_sign'] == 1]
specific_frame2 = specific_frame.loc[specific_frame['dollar_sign'] == 2]
specific_frame3 = specific_frame.loc[specific_frame['dollar_sign'] == 3]
specific_frame4 = specific_frame.loc[specific_frame['dollar_sign'] == 4]

In [22]:
name1 = list(specific_frame1["name"])
lat1 = list(specific_frame1["latitude"])
lon1 = list(specific_frame1["longitude"])

name2 = list(specific_frame2["name"])
lat2 = list(specific_frame2["latitude"])
lon2 = list(specific_frame2["longitude"])

name3 = list(specific_frame3["name"])
lat3 = list(specific_frame3["latitude"])
lon3 = list(specific_frame3["longitude"])

name4 = list(specific_frame4["name"])
lat4 = list(specific_frame4["latitude"])
lon4 = list(specific_frame4["longitude"])

In [23]:
fg1 = folium.FeatureGroup(name="1 Dollar Sign")
fg2 = folium.FeatureGroup(name="2 Dollar Signs")
fg3 = folium.FeatureGroup(name="3 Dollar Signs")
fg4 = folium.FeatureGroup(name="4 Dollar Signs")

In [24]:
for lt, ln, nm in zip(lat1, lon1, name1):
    fg1.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(1)+" price",
    fill_color='red', fill=True,  color = 'red', fill_opacity=0.7))
    
for lt, ln, nm in zip(lat2, lon2, name2):
    fg2.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(2)+" price",
    fill_color='orange', fill=True,  color = 'orange', fill_opacity=0.7))
    
for lt, ln, nm in zip(lat3, lon3, name3):
    fg3.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(3)+" price",
    fill_color='yellow', fill=True,  color = 'yellow', fill_opacity=0.7))
    
for lt, ln, nm in zip(lat4, lon4, name4):
    fg4.add_child(folium.CircleMarker(location=[lt, ln], radius = 1, popup= nm + ", " + str(4)+" price",
    fill_color='green', fill=True,  color = 'green', fill_opacity=0.7))

In [25]:
m2 = folium.Map([34, -112], zoom_start=8)

In [26]:
m2.add_child(fg1)
m2.add_child(fg2)
m2.add_child(fg3)
m2.add_child(fg4)

If you want to combine them, add_child to the same map object that was created for the Chloropleth

In [27]:
m.add_child(fg1)
m.add_child(fg2)
m.add_child(fg3)
m.add_child(fg4)

Add "Layer Control", which allows for the different points to be toggled on and off

In [ ]:
m.add_child(folium.LayerControl())

In [ ]:
m.save(outfile='arizona_plots_and_chloro.html')